In [40]:
import pandas as pd
import sqlite3, requests

import re

In [41]:
conn = sqlite3.connect('D:\\PYTHON\\randewoo\\database.db')
df = pd.read_sql_query('SELECT * FROM items_data', conn)
conn.close()

In [42]:
# df = df.applymap(lambda x: x.replace("Парфюмерная вода женские", "Парфюмерная вода женская") if isinstance(x, str) and "Парфюмерная вода женские" in x else x)
# df = df.applymap(lambda x: x.replace("Парфюмерная вода унисекс", "Парфюмерная вода женская") if isinstance(x, str) and "Парфюмерная вода унисекс" in x else x)
# df = df.applymap(lambda x: x.replace("Парфюмерная вода мужские", "Парфюмерная вода мужская") if isinstance(x, str) and "Парфюмерная вода мужские" in x else x)

In [43]:
df = df.drop('search_name', axis=1)

In [45]:
out_df = df[(df['commerce_type'] == 'Духи унисекс') | (df['commerce_type'] == 'Духи женские')]

In [46]:
out_df.columns

Index(['art', 'item_name', 'price', 'prev_price', 'nds', 'commerce_type',
       'weight', 'width', 'height', 'length', 'image_url', 'item_type',
       'model_name', 'brand', 'sex', 'add_together', 'volume', 'country',
       'target_audience', 'composition', 'danger_class', 'expiration_date',
       'okpd_code', 'annotation'],
      dtype='object')

In [47]:
def compare_strings_url(s1, s2):
    s1 = pd.Series([x for x in str(s1).lower().split(",") if 'мл' not in x])
    s2 = pd.Series([x for x in str(s2).lower().split(",") if 'мл' not in x])
    
    s1 = pd.Series([x for x in s1.str.split().explode() if x != ''])
    s2 = pd.Series([x for x in s2.str.split().explode() if x != ''])
    if len(s1) < len(s2):
        shorter_series, longer_series = s1, s2
    else:
        shorter_series, longer_series = s2, s1

    count = shorter_series.isin(longer_series).sum()

    if count >= (len(shorter_series)):
        return 1
    else:
        return 0

def fill_img_url(row):
    if pd.isnull(row['image_url']):
        query = row['item_name']
        url = f'https://api.retailrocket.ru/api/2.0/recommendation/Search/525a65850d422d408c357a8c/?&phrase={query}session=64a1b414b2113a5f4a3e6c8d&pvid=605539862619794&isDebug=false&format=json'
        response = requests.get(url=url).json()[0]
        if compare_strings_url(row['item_name'], f'''{response['Params']['brand']}, {response['Params']['name_part_1']}''') == 1:
            return response['PictureUrl']
        else:
            return row['image_url']
    else:
        return row['image_url']

In [48]:
def compare_strings(s1, s2):
    s1 = pd.Series([x for x in str(s1).lower().split(",") if 'мл' not in x])
    s2 = pd.Series([x for x in str(s2).lower().split(",") if 'мл' not in x])
    
    s1 = pd.Series([x for x in s1.str.split().explode() if x != ''])
    s2 = pd.Series([x for x in s2.str.split().explode() if x != ''])

    if len(s1) == len(s2):
        shorter_series, longer_series = s1, s2
    else:
        return 0

    count = shorter_series.isin(longer_series).sum()

    if count >= (len(shorter_series)):
        return 1
    else:
        return 0

unique_id = 1000000
for index, row in out_df.iterrows():
    item_name = row['item_name']
    if int(row['add_together']) < 999999: 
        similar_items = out_df[out_df['item_name'].apply(lambda x: compare_strings(item_name, x)) == 1]
        out_df.loc[similar_items.index, 'add_together'] = unique_id
        unique_id += 1

In [49]:
# def compare_strings(s1, s2):
#     s1 = pd.Series([x for x in str(s1).lower().split(",") if 'мл' not in x])
#     s2 = pd.Series([x for x in str(s2).lower().split(",") if 'мл' not in x])
    
#     s1 = pd.Series([x for x in s1.str.split().explode() if x != ''])
#     s2 = pd.Series([x for x in s2.str.split().explode() if x != ''])

#     if len(s1) < len(s2):
#         shorter_series, longer_series = s1, s2
#     else:
#         shorter_series, longer_series = s2, s1

#     count = shorter_series.isin(longer_series).sum()

#     if count >= (len(shorter_series)) and count > 3:
#         return 1
#     else:
#         return 0

# unique_id = 1000000
# for index, row in out_df.iterrows():
#     item_name = row['item_name']
#     if int(row['add_together']) < 999999: 
#         similar_items = out_df[out_df['item_name'].apply(lambda x: compare_strings(item_name, x)) == 1]
#         out_df.loc[similar_items.index, 'add_together'] = unique_id
#         unique_id += 1

In [50]:
out_df['image_url'] = out_df.apply(fill_img_url, axis=1)

C:\Users\birysa\AppData\Local\Temp\ipykernel_2376\1415567431.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  out_df['image_url'] = out_df.apply(fill_img_url, axis=1)


In [51]:
out_df['image_url']

1       http://89.108.75.237/images/categories/goods_f...
2                 https://cdn2.randewoo.ru/img/419609/s/1
20      http://89.108.75.237/images/categories/goods_f...
63                https://cdn2.randewoo.ru/img/140545/n/1
64                https://cdn2.randewoo.ru/img/140542/n/1
                              ...                        
3754    http://89.108.75.237/images/categories/goods_f...
3755    http://89.108.75.237/images/categories/goods_f...
3758              https://cdn2.randewoo.ru/img/346282/s/1
3761              https://cdn2.randewoo.ru/img/408248/n/1
3790              https://cdn2.randewoo.ru/img/328153/s/1
Name: image_url, Length: 325, dtype: object

In [52]:
out_df.insert(5, 'ozon_id', '')

In [53]:
out_df.insert(7, 'barcode', '')

In [54]:
out_df.insert(13, 'add_photo', '')
out_df.insert(14, '360_photo', '')
out_df.insert(15, 'art_photo', '')

In [55]:
out_df = out_df.drop('model_name', axis=1)

In [56]:
columns = list(out_df.columns)

item_type_index = columns.index('item_type')
brand_index = columns.index('brand')

columns[item_type_index], columns[brand_index] = columns[brand_index], columns[item_type_index]

out_df = out_df.reindex(columns=columns)

In [57]:
item_type_index = columns.index('add_together')
brand_index = columns.index('sex')

columns[item_type_index], columns[brand_index] = columns[brand_index], columns[item_type_index]

out_df = out_df.reindex(columns=columns)

In [58]:
item_type_index = columns.index('sex')
brand_index = columns.index('volume')

columns[item_type_index], columns[brand_index] = columns[brand_index], columns[item_type_index]

out_df = out_df.reindex(columns=columns)

In [59]:
import numpy as np

out_df.insert(20, 'in_one_item', np.ones(len(out_df), dtype=int))

In [60]:
item_type_index = columns.index('danger_class')
brand_index = columns.index('target_audience')

columns[item_type_index], columns[brand_index] = columns[brand_index], columns[item_type_index]

out_df = out_df.reindex(columns=columns)

In [61]:
out_df.insert(23, 'rich', '')
out_df.insert(24, 'model_name', '')
out_df.insert(25, 'min_kid', '')
out_df.insert(26, 'max_kid', '')
out_df.insert(27, 'plan', '')

In [62]:
item_type_index = columns.index('annotation')
brand_index = columns.index('composition')

columns[item_type_index], columns[brand_index] = columns[brand_index], columns[item_type_index]

out_df = out_df.reindex(columns=columns)

In [63]:
search_str = ["ZARKOPERFUME", "Acqua di Parma", "Abercrombie & Fitch", "Aerin", "Giorgio Armani", "Armani", "Amouage", "Balmain", "Benetton", "Boadicea The Victorious", "BYBOZO", "Byredo", "Carolina Herrera", "Calvin Klein", "CK", "Creed", "Diptyque", "Donna Karan", "Christian Dior", "Ermenegildo Zegna", "Etat Libre D`Orange", "Etat", "Evody", "Ex Nihilo", "Franck Olivier", "Giardino Benessere", "Guess", "Gucci", "Hermes", "Hermetica", "Histoires de Parfums", "Jacques Zolty", "Jeanne Arthes", "John Varvatos", "Juicy Couture", "Couture", "Juliette has a Gun", "Lanvin", "Lacoste", "Lancome", "Les Contes", "Michael Kors", "Moresque", "Olfactive Studio", "Paco Rabanne", "Penhaligon's", "Philipp Plein", "Ralph Lauren", "Stella McCartney", "Tiziana Terenzi", "Tom Ford", "Tommy Hilfiger", "Tous", "V Canto", "Vilhelm Parfumerie", "Zarkoperfume", "Versace", "Givenchy", "Memo", "Hermes", "Jo Malone", 'аква ди парма', 'аберкромби & фитч', 'аерин', 'Амуаж', 'Бальмен', 'Бальмэйн', 'Бенеттон', 'Бодичеа Викториос', 'Байбозо', 'Биредо', 'Каролина Херрера', 'Кельвин Кляйн', 'Крид', 'Диптик', 'Донна Каран', 'Диор', 'Ерменегильдо Зегна', 'Этат', 'Эводи', 'Экс Нихило', 'Фрэнк Оливер', 'Жардино Бенессере', 'Гесс', 'Гуччи', 'Гермес', 'Герметика', 'Хисторес де парфюмс', 'Жакье Золти', 'Жан Артес', 'Джон Варватос', 'Джуси Кутюр', 'Майкл Корс', 'Мореск', 'Ольфактив Студио', 'Пако Рабанне', 'Пенхалигонс', 'Филипп Пляйн', 'Ральф Лорен', 'Стелла МакКартни', 'Тизмана Терензи', 'Том Форд', 'Томми хилфигер', 'Той', 'В Канто', 'Вильгельм Парфюмерие', 'Заркоперфюм', 'Версаче', 'Дживанши', 'Мемо', 'Гермес', 'Джо Малон']

In [64]:
for item in search_str:
    mask = out_df['brand'].astype(str).apply(lambda y: bool(re.search(re.escape(item.lower()), y.lower(), re.IGNORECASE)))
    out_df.loc[mask, 'brand'] = "НЕТ БРЕНДА"

In [65]:
delete_str = ["Antonio Banderas", "Hugo Boss", "Carolina Herrera", "Chanel", "Clarins", "Dior", "Dolce & Gabbana", "Escentric Molecules", "Armani", "Givenchy", "Lancome", "Lanvin", "Versace", 'Антонио Бандерас', 'Хуго Босс', 'Каролина Херрера', 'Шанел', 'Кларинс', 'Диор', 'Дольче & Габанна', 'Ескентрик Молекулес', 'Армани', 'Дживанши', 'Ланкоме', 'Ланвин', 'Версаче', 'Хьюго Босс', 'Клэринс', 'Клеринс', 'Дольче Габана', 'Дольче Габбана', 'Живанши', 'Ланком']

In [66]:
for item in delete_str:
    mask = out_df.apply(lambda row: row.astype(str).str.contains(r'\b' + re.escape(item) + r'\b', case=False, regex=True).any(), axis=1)
    out_df = out_df[~mask]

In [67]:
replace_str = ''
for item in search_str:
    mask = out_df['item_name'].astype(str).apply(lambda y: bool(re.search(r'\b' + re.escape(item.lower()) + r'\b' + ', ', y.lower(), re.IGNORECASE)))
    out_df.loc[mask, 'item_name'] = out_df.loc[mask, 'item_name'].apply(lambda x: re.sub(r'\b' + re.escape(item) + r'\b' + ', ', replace_str, x, flags=re.IGNORECASE) if isinstance(x, str) else x)

In [68]:
for item in search_str:
    mask = out_df['annotation'].astype(str).apply(lambda y: bool(re.search(r'\b' + re.escape(item.lower()) + r'\b', y.lower(), re.IGNORECASE)))
    out_df.loc[mask, 'annotation'] = out_df.loc[mask, 'annotation'].apply(lambda x: re.sub(r'\b' + re.escape(item) + r'\b', replace_str, x, flags=re.IGNORECASE) if isinstance(x, str) else x)

In [69]:
out_df.insert(20, 'in_one_item', np.ones(len(out_df), dtype=int))

In [70]:
out_df = out_df.applymap(lambda x: "" if isinstance(x, str) and ".svg" in x else x)
out_df = out_df.applymap(lambda x: x.replace('s/1', 'z/1') if isinstance(x, str) and 's/1' in x else x)
out_df = out_df.applymap(lambda x: x.replace('n/1', 'z/1') if isinstance(x, str) and 'n/1' in x else x)

In [71]:
out_df = out_df.sort_values('image_url', na_position='last')
out_df = out_df.drop_duplicates(subset=[out_df.columns[0]], keep='first')
out_df = out_df.drop_duplicates(subset=[out_df.columns[1]], keep='first')

In [72]:
out_df = out_df.iloc[3:]
out_df = out_df[(out_df['volume'].astype(int) >= 14) & (out_df['volume'].astype(int) <= 200)]

In [73]:
countries = ["Австралия", "Австрия", "Азербайджан", "Аргентина", "Бахрейн", "Бельгия", "Великобритания", "Германия", "Греция", "Дания", "Египет", "Индия", "Ирландия", "Исландия", "Испания", "Италия", "Канада", "Катар", "Китай", "Кувейт", "Латвия", "Ливан", "Монако", "Нидерланды", "ОАЭ", "Оман", "Польша", "Португалия", "Россия", "США", "Саудовская Аравия", "Сирия", "Словакия", "Словения", "Турция", "Украина", "Финляндия", "Франция", "Чехия", "Швейцария", "Швеция", "Япония"]

out_df.loc[~out_df['country'].isin(countries), 'country'] = ''

In [74]:
import re

def replace_ml(text):
    return re.sub(r'(\d+)мл', r'\1 мл', text)

out_df = out_df.applymap(lambda x: replace_ml(x) if isinstance(x, str) else x)


In [75]:
# def compare_strings(s1, s2):
#     s1 = pd.Series([x for x in str(s1).lower().split(",") if 'мл' not in x])
#     s2 = pd.Series([x for x in str(s2).lower().split(",") if 'мл' not in x])
    
#     s1 = pd.Series([x for x in s1.str.split().explode() if x != ''])
#     s2 = pd.Series([x for x in s2.str.split().explode() if x != ''])

#     if len(s1) == len(s2):
#         shorter_series, longer_series = s1, s2
#     else:
#         return 0

#     count = shorter_series.isin(longer_series).sum()

#     if count >= (len(shorter_series)) and count > 3:
#         return 1
#     else:
#         return 0

In [76]:
# unique_id = 1000000
# for index, row in out_df.iterrows():
#     item_name = row['item_name']
#     if int(row['add_together']) < 999999: 
#         similar_items = out_df[out_df['item_name'].apply(lambda x: compare_strings(item_name, x)) == 1]
#         out_df.loc[similar_items.index, 'add_together'] = unique_id
#         unique_id += 1

In [77]:
out_df['item_type'] = 'Духи'

In [78]:
file_path = '~/Desktop/Духи женские.xlsx'

out_df.sort_values('add_together').to_excel(file_path, index=False)